In [41]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
from PIL import Image
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
data = pd.read_csv("~/code/joaonpmsantos/Ocular_Disease_Recognition/Data_Analysis/df_binary.csv")

In [3]:
data['target_D'] = (data['labels'] != "['D']").astype('int')

In [61]:
data_path = "code/joaonpmsantos/Ocular_Disease_Recognition/raw_data/preprocessed_images"
imgs = []
images_path = [elt for elt in os.listdir(os.path.join(Path.home(),data_path)) if elt.find('.jpg')>0]
for file in data.filename[:1000]:
    path = os.path.join(Path.home(),data_path, file)
    if os.path.exists(path):
        image = Image.open(path)
        imgs.append(np.array(image))

In [62]:
np.array(imgs).shape

(1000, 512, 512, 3)

In [63]:
X_train, X_test, y_train, y_test = train_test_split(np.array(imgs), data.target_D[:1000], train_size=0.7)

In [75]:
def load_own_model():
  model = models.Sequential()
  model.add(layers.Rescaling(1/255, input_shape = (512,512,3)))
  model.add(layers.Conv2D(64, (5,5), activation="relu", padding="same", strides=(1,1)))
  model.add(layers.Conv2D(64, (4,4), activation="relu", padding="same", strides=(1,1)))
  model.add(layers.MaxPool2D(pool_size=(2,2)))

  model.add(layers.Conv2D(128, (3,3), activation="relu", padding="same", strides=(1,1)))
  model.add(layers.Conv2D(128, (3,3), activation="relu", padding="same", strides=(2,2)))
  model.add(layers.MaxPool2D(pool_size=(2,2)))

  model.add(layers.Conv2D(512, (3,3), activation="relu", padding="same", strides=(1,1)))
  model.add(layers.Conv2D(512, (3,3), activation="relu", padding="same", strides=(1,1)))
  model.add(layers.Conv2D(512, (2,2), activation="relu", padding="same", strides=(2,2)))
  model.add(layers.MaxPool2D(pool_size=(2,2)))

  model.add(layers.Conv2D(512, (2,2), activation="relu", padding="same", strides=(1,1)))
  model.add(layers.Conv2D(512, (2,2), activation="relu", padding="same", strides=(1,1)))
  model.add(layers.MaxPool2D(pool_size=(2,2)))
  
  model.add(layers.Flatten())
  model.add(layers.Dense(512, activation="relu"))
  model.add(layers.Dense(126, activation="relu"))
  model.add(layers.Dense(64, activation="relu"))
  model.add(layers.Dense(1, activation="sigmoid"))
  
  model.compile(optimizer="adam",
                loss="binary_crossentropy",
                metrics=["accuracy","Recall"])
  return model

In [65]:
y_train.shape

(700,)

In [66]:
len(np.where(y_train == 0)[0])/len(y_train)

0.24285714285714285

In [67]:
class_weights = class_weight.compute_class_weight('balanced',classes=np.unique(y_train), y=y_train)

In [68]:
np.unique(y_train)

array([0, 1])

In [69]:
class_weights

array([2.05882353, 0.66037736])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    featurewise_center = False,
    featurewise_std_normalization = False,
    rotation_range = 10,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    horizontal_flip = True,
    zoom_range = (0.8, 1.2),
    ) 

datagen.fit(X_train[np.where(y_train == 0)[0]])
datagen

In [76]:
model =load_own_model()

In [77]:
model.fit(X_train, y_train, epochs=70, 
          batch_size=32, validation_split=0.3, callbacks=[EarlyStopping(patience=10, restore_best_weights=True)], 
          class_weight={0:class_weights[0],1:class_weights[1]})

Epoch 1/70
16/16 [==============================] - 494s 31s/step - loss: 0.6807 - accuracy: 0.3313 - recall: 0.2143 - val_loss: 0.6898 - val_accuracy: 0.7204 - val_recall: 1.0000
Epoch 2/70
16/16 [==============================] - 505s 32s/step - loss: 0.6777 - accuracy: 0.7730 - recall: 1.0000 - val_loss: 0.6812 - val_accuracy: 0.7204 - val_recall: 1.0000
Epoch 3/70
16/16 [==============================] - 540s 34s/step - loss: 0.6777 - accuracy: 0.7730 - recall: 1.0000 - val_loss: 0.6773 - val_accuracy: 0.7204 - val_recall: 1.0000
Epoch 4/70
16/16 [==============================] - 511s 32s/step - loss: 0.6770 - accuracy: 0.7730 - recall: 1.0000 - val_loss: 0.6829 - val_accuracy: 0.7204 - val_recall: 1.0000
Epoch 5/70
16/16 [==============================] - 534s 34s/step - loss: 0.6772 - accuracy: 0.7730 - recall: 1.0000 - val_loss: 0.6822 - val_accuracy: 0.7204 - val_recall: 1.0000
Epoch 6/70
16/16 [==============================] - 570s 36s/step - loss: 0.6772 - accuracy: 0.7730 